# Video Game Recommender Project

## Part 1: Getting the Data

I decided to retrieve the data from an API database curated by IGDB (https://www.igdb.com/). The database holds over 250 000 games which is a great starting point to make a video game recommender off of.

In [1]:
import requests
import pandas as pd

To access the API, I had to create an account on the Twitch Developer Portal which allowed me to a generate a client ID and a client secret. Using these credentials, I had generate an access token which would allow me to make post request to the server which the database belongs to so i can then retrieve the data. 

In [28]:
token_endpoint = "https://id.twitch.tv/oauth2/token"
client_id = "8m485qqc89htq84deio57c7tryri4v"
client_secret = "6ci2zwn9i9seth4ldmpwf053hcvoj9"

# Define the parameters for the request
params = {
    "grant_type": "client_credentials",
    "client_id": client_id,
    "client_secret": client_secret
}

# Make the request to obtain access token
response = requests.post(token_endpoint, data=params)

# Parse the response
if response.status_code == 200:
    access_token = response.json().get("access_token")
    print("Access Token:", access_token)
else:
    print("Failed to get access token. Status code:", response.status_code)

Access Token: o9ut9dw6c0m41sfwysxv9ig3lerks7


Finally, I have all the credentials I need to access the API, so now I can make get requests to the server for the data. The following script initializes an empty list to store all the retrieved games and sets the initial query parameters such as the fields to retrieve and the limit of games per request. The script iterates through the API requests, adding the retrieved games to the list until a specified cap is reached. It utilizes the requests library to make GET requests with the appropriate headers for authentication. If successful, the response is converted to JSON and the games are added to the list. The loop continues until either all available games are fetched or there are no more games to retrieve. Finally, the script trims the list to match the cap and creates a DataFrame using pandas to organize the collected games data for further analysis.

In [4]:
# API Credentials
client_id = "8m485qqc89htq84deio57c7tryri4v"
client_secret = "6ci2zwn9i9seth4ldmpwf053hcvoj9"
access_token = 'ok2zlmcqawhs5qsxk3nolcsupe51nl'

# API Endpoint for fetching games
url = "https://api.igdb.com/v4/games"

# Initialize an empty list to store all games
all_games = []

# Define initial query parameters
params = {
    "fields": "*",
    "limit": 500,  # Adjust as needed
    "offset": 0   # Start from the beginning
}

# Include your access token in the headers for authentication
headers = {
    "Client-ID": client_id,
    "Authorization": f"Bearer {access_token}"
}

# Set a cap for the total number of games
cap_games = 10000000

# Make the GET requests until the cap is reached
while len(all_games) < cap_games:
    response = requests.get(url, headers=headers, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Convert the response to JSON
        games_data = response.json()
        
        # Add the retrieved games to the list
        all_games.extend(games_data)
        
        # Check if there are more games to fetch
        if len(games_data) < 50:
            break  # Exit the loop if there are no more games to fetch
        else:
            # Increment the offset to fetch the next page of results
            params["offset"] += 50
    else:
        print("Failed to retrieve data:", response.status_code)
        break

# Trim the list to match the cap
all_games = all_games[:cap_games]

# Create DataFrame from the collected games data
games_df = pd.DataFrame(all_games)

# Now you have all your data in a DataFrame
games_df

,id,age_ratings,alternative_names,category,cover,created_at,external_games,first_release_date,game_modes,genres,...,aggregated_rating_count,ports,dlcs,remakes,franchise,expansions,standalone_expansions,forks,remasters,expanded_games
0,131913,[101730],"[40085, 40086, 40087]",0,267633.0,1584788069,[1977891],1.474416e+09,[1],[34],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,88308,NaN,NaN,0,64849.0,1519237439,"[243459, 1960113]",1.478650e+09,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,63308,[14739],NaN,0,78658.0,1505087910,"[12596, 118080, 1185144]",1.372810e+09,"[2, 3]",[5],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,95080,NaN,NaN,0,NaN,1521818623,[1989881],NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,104748,NaN,NaN,0,NaN,1530519587,"[1155919, 1972586]",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2740745,270311,"[155339, 156812, 156813, 156814, 156815, 156816]",NaN,0,338712.0,1696471845,"[2749772, 2749823, 2749915, 2749948, 2749957, ...",1.697674e+09,[1],[13],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2740746,26778,"[12738, 12739, 74252]","[23222, 103500, 103531]",0,345907.0,1484338599,"[46568, 148728, 215876, 246558, 402671, 459935...",5.311872e+08,[1],"[8, 25, 31]",...,NaN,[109597],NaN,[134561],NaN,NaN,NaN,NaN,NaN,NaN
2740747,287107,[167130],NaN,0,NaN,1708046589,"[2888238, 2888279, 2888301]",1.707782e+09,NaN,[12],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2740748,172430,"[46359, 55966, 95862, 118055]",[102314],8,180918.0,1632435987,"[2124213, 2124220, 2124674, 2135085, 2160840, ...",1.632355e+09,[1],"[8, 12, 13, 15, 25]",...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The dataframe has over 2.5 million rows when the webpage says it has over 250 000 game. That means there is a lot of data that needs to be dropped before saving. I dont want to save a dataset with over 2.5 million rows. I will drop the duplicates and then save it as a csv. We will end up with just over 250 000 games.

In [7]:
games_df.drop_duplicates(subset=['id'], inplace=True)
games_df

,id,age_ratings,alternative_names,category,cover,created_at,external_games,first_release_date,game_modes,genres,...,aggregated_rating_count,ports,dlcs,remakes,franchise,expansions,standalone_expansions,forks,remasters,expanded_games
0,131913,[101730],"[40085, 40086, 40087]",0,267633.0,1584788069,[1977891],1.474416e+09,[1],[34],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,88308,NaN,NaN,0,64849.0,1519237439,"[243459, 1960113]",1.478650e+09,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,63308,[14739],NaN,0,78658.0,1505087910,"[12596, 118080, 1185144]",1.372810e+09,"[2, 3]",[5],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,95080,NaN,NaN,0,NaN,1521818623,[1989881],NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,104748,NaN,NaN,0,NaN,1530519587,"[1155919, 1972586]",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2738495,270311,"[155339, 156812, 156813, 156814, 156815, 156816]",NaN,0,338712.0,1696471845,"[2749772, 2749823, 2749915, 2749948, 2749957, ...",1.697674e+09,[1],[13],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2738496,26778,"[12738, 12739, 74252]","[23222, 103500, 103531]",0,345907.0,1484338599,"[46568, 148728, 215876, 246558, 402671, 459935...",5.311872e+08,[1],"[8, 25, 31]",...,NaN,[109597],NaN,[134561],NaN,NaN,NaN,NaN,NaN,NaN
2738497,287107,[167130],NaN,0,NaN,1708046589,"[2888238, 2888279, 2888301]",1.707782e+09,NaN,[12],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2738498,172430,"[46359, 55966, 95862, 118055]",[102314],8,180918.0,1632435987,"[2124213, 2124220, 2124674, 2135085, 2160840, ...",1.632355e+09,[1],"[8, 12, 13, 15, 25]",...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
games_df.to_csv('../python lessons/''games.csv')